
# **DCGAN (Robust) — Colab Fast FID, EMA, Mixed Precision**

A strong baseline **DCGAN** (convolutional) tuned for stability and decent performance:
- Non-saturating GAN loss (logistic)
- **Adam** (β1=0.5, β2=0.999) per DCGAN paper
- **TTUR** (optional via separate G/D LRs; defaults equal)
- **BatchNorm** in G; **no BN in last layer**; D uses **LeakyReLU + Dropout**
- **He/Normal (0.02)** initialization (as in DCGAN)
- **EMA** of G for sampling + FID
- **Mixed precision** (on supported GPUs) + **XLA** (train steps), FID kept in float32
- 64×64 pipeline, persistent **4×4** grids (~330 px) with epoch/time overlay
- **Fast FID** (cached real stats), checkpoints, CSV logging, **loss & FID plots**
- GIF of training grids, GIF of FID, and **side-by-side MP4** (Slides friendly)


In [ ]:

# =====================
# Configuration
# =====================
import os

# Data & output
DATA_DIR            = "monet_jpg"  # Place JPG/PNG images here
OUTPUT_DIR          = "outputs_dcgan_robust"
SAMPLES_DIR         = os.path.join(OUTPUT_DIR, "samples")
CKPT_DIR            = os.path.join(OUTPUT_DIR, "checkpoints")
METRICS_CSV         = os.path.join(OUTPUT_DIR, "metrics_history.csv")

# Image & training
IMG_SIZE            = 64
CHANNELS            = 3
BATCH_SIZE          = 64
BUFFER_SIZE         = 4000
EPOCHS              = 100

# Work per epoch
REPEAT_DATASET      = True
STEPS_PER_EPOCH     = 300

# Optim
GEN_LR              = 2e-4
DISC_LR             = 2e-4
BETA_1              = 0.5
BETA_2              = 0.999

# Label smoothing (mild) & optional instance noise
REAL_LABEL_MIN      = 0.90
REAL_LABEL_MAX      = 1.00
FAKE_LABEL_MIN      = 0.00
FAKE_LABEL_MAX      = 0.10
INSTANCE_NOISE_START= 0.00     # 0 to disable by default
INSTANCE_NOISE_END  = 0.00

# Display / saving
SAMPLES             = 16
GRID_NROW, GRID_NCOL= 4, 4
DISPLAY_WIDTH_PX    = 330
SAVE_GRID_EVERY     = 1

# Caching
CACHE_DATASET       = True
CACHE_PATH          = ""

# FID (fast & memory-safe)
COMPUTE_FID_EVERY   = 5
REAL_FID_SAMPLES    = 1000
FAKE_FID_SAMPLES    = 500
FID_GEN_BATCH       = 128
FID_ACT_BATCH       = 1024

# Precision
FORCE_FLOAT32_TRAIN = False

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SAMPLES_DIR, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)


In [ ]:

# =====================
# Imports & Environment
# =====================
import glob, time, math, shutil
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


In [ ]:

# =====================
# Mixed Precision + XLA
# =====================
from tensorflow.keras import mixed_precision

gpus = tf.config.list_physical_devices('GPU')
gpu_name = ""
if gpus:
    try:
        details = tf.config.experimental.get_device_details(gpus[0])
        gpu_name = details.get('device_name', '')
    except Exception:
        pass

if not FORCE_FLOAT32_TRAIN and gpus and any(x in (gpu_name or '').upper() for x in ["A100","A10","L4","T4","V100","P100","TITAN"]):
    mixed_precision.set_global_policy('mixed_float16')
else:
    mixed_precision.set_global_policy('float32')

try:
    tf.config.optimizer.set_jit(True)
    xla_state = "ON"
except Exception:
    xla_state = "OFF"

print("Policy:", mixed_precision.global_policy())
print("XLA JIT:", xla_state, "| GPU:", gpu_name or "None")


In [ ]:

# =====================
# Single-Device Strategy
# =====================
strategy = tf.distribute.get_strategy()
print("Using default single-device strategy.")



### (Optional) Copy dataset from Drive to local disk (faster I/O in Colab)
Uncomment and run once if your images are in Drive.


In [ ]:

# !rsync -ah --info=progress2 "/content/drive/MyDrive/monet_jpg/" "/content/monet_jpg/"
# DATA_DIR = "/content/monet_jpg"


In [ ]:

# =====================
# Data Loader (fast JPEG path + simple flip aug)
# =====================
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path):
    img_bytes = tf.io.read_file(path)
    is_jpeg = tf.strings.regex_full_match(tf.strings.lower(path), ".*\.(jpg|jpeg)$")
    def _decode_jpeg(): return tf.io.decode_jpeg(img_bytes, channels=3, dct_method='INTEGER_ACCURATE')
    def _decode_any():  return tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.cond(is_jpeg, _decode_jpeg, _decode_any)
    img.set_shape([None, None, 3])
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]
    h = tf.shape(img)[0]; w = tf.shape(img)[1]
    side = tf.minimum(h, w)
    img = tf.image.crop_to_bounding_box(img, (h-side)//2, (w-side)//2, side, side)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), method="bicubic")
    img = tf.image.random_flip_left_right(img)  # light aug outside train step
    img = (img - 0.5) * 2.0                     # [-1,1]
    return img

def make_dataset(data_dir, batch_size):
    patterns = ["*.jpg","*.jpeg","*.png"]
    files = []
    for p in patterns:
        files.extend(glob.glob(os.path.join(data_dir, p)))
    if not files:
        raise FileNotFoundError(f"No images found under '{data_dir}'. Place JPG/PNG files there.")
    paths = tf.constant(files)
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    if CACHE_DATASET:
        ds = ds.cache(CACHE_PATH) if CACHE_PATH else ds.cache()
    ds = ds.shuffle(min(len(files), BUFFER_SIZE), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size, drop_remainder=True)
    if REPEAT_DATASET:
        ds = ds.repeat()
    ds = ds.prefetch(AUTOTUNE)
    return ds, len(files)

train_dataset, num_files = make_dataset(DATA_DIR, BATCH_SIZE)
print(f"Found {num_files} images; batch size {BATCH_SIZE}; IMG_SIZE {IMG_SIZE}.")


In [ ]:

# =====================
# FID Utilities (GPU Inception in float32)
# =====================
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import mixed_precision

try:
    from scipy.linalg import sqrtm as _scipy_sqrtm
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

_prev = mixed_precision.global_policy()
mixed_precision.set_global_policy('float32')
_dev = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
with tf.device(_dev):
    _inception = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=(299,299,3))
_inception.trainable = False
mixed_precision.set_global_policy(_prev)

REAL_STATS_CACHE = None

def _to_uint8(x01):
    x = tf.clip_by_value(x01, 0.0, 1.0)
    return tf.cast(x * 255.0, tf.uint8)

def _prep(uint8_batch):
    x = tf.cast(uint8_batch, tf.float32)
    x = tf.image.resize(x, (299,299), method='bicubic')
    return preprocess_input(x)

@tf.function(jit_compile=False)
def _inception_acts(x_f32):
    return _inception(x_f32, training=False)

def _activations_batched(uint8_images, batch_size=FID_ACT_BATCH):
    n = int(uint8_images.shape[0])
    outs = []
    dev = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
    for i in range(0, n, batch_size):
        x = _prep(uint8_images[i:i+batch_size])
        with tf.device(dev):
            acts = _inception_acts(x)
        outs.append(acts)
    return tf.concat(outs, axis=0)

def _mu_sigma(acts: tf.Tensor):
    a = acts.numpy()
    return a.mean(0), np.cov(a, rowvar=False)

def _fid(mu1, s1, mu2, s2):
    if _HAS_SCIPY:
        covmean = _scipy_sqrtm(s1.dot(s2)).real
    else:
        covmean = tf.linalg.sqrtm(tf.convert_to_tensor(s1 @ s2, tf.float64))
        covmean = tf.math.real(covmean).numpy()
    diff = mu1 - mu2
    return float(diff.dot(diff) + np.trace(s1 + s2 - 2.0 * covmean))

def sample_real_uint8(ds, n):
    out, cnt = [], 0
    for b in ds:
        x = (b + 1.0) * 0.5
        out.append(_to_uint8(x)); cnt += x.shape[0]
        if cnt >= n: break
    return tf.concat(out, 0)[:n]


In [ ]:

# =====================
# Sampling Utilities (4x4 display)
# =====================
from IPython.display import display, Image as IPyImage

def make_grid(imgs, nrow, ncol):
    rows = []
    for r in range(nrow):
        row = np.concatenate([imgs[r*ncol + c] for c in range(ncol)], axis=1)
        rows.append(row)
    return np.concatenate(rows, axis=0)

def save_and_show_grid(imgs_01, epoch, secs=None, nrow=GRID_NROW, ncol=GRID_NCOL, out_dir=SAMPLES_DIR, prefix=""):
    grid = make_grid(imgs_01, nrow, ncol)
    os.makedirs(out_dir, exist_ok=True)
    raw_path = os.path.join(out_dir, f"{prefix}epoch_{epoch:04d}.png")
    imageio.imwrite(raw_path, (grid * 255).astype(np.uint8))

    fig = plt.figure(figsize=(2.2, 2.2), dpi=150)   # ~330 px
    plt.imshow(grid)
    title = f"Epoch {epoch}" + (f" | {secs:.1f}s" if secs is not None else "")
    plt.title(title, fontsize=8)
    plt.axis('off')
    ann_path = os.path.join(out_dir, f"{prefix}epoch_{epoch:04d}_annotated.png")
    plt.savefig(ann_path, bbox_inches='tight', pad_inches=0.02)
    plt.close(fig)
    display(IPyImage(filename=ann_path, width=DISPLAY_WIDTH_PX))
    return raw_path, ann_path


In [ ]:

# =====================
# DCGAN Models
# =====================
WEIGHT_INIT = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

LATENT_DIM = 128
BASE_CH    = 64  # feature maps base

def make_generator(img_size=IMG_SIZE, channels=CHANNELS, base_ch=BASE_CH, latent_dim=LATENT_DIM):
    assert img_size == 64, "This template expects IMG_SIZE=64 (change model if needed)."
    z_in = layers.Input(shape=(latent_dim,))

    # project & reshape to 4x4x(8*base)
    n_ch = base_ch * 8  # 512
    x = layers.Dense(4*4*n_ch, use_bias=False, kernel_initializer=WEIGHT_INIT)(z_in)
    x = layers.Reshape((4,4,n_ch))(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)

    # 4->8
    x = layers.Conv2DTranspose(base_ch*4, kernel_size=4, strides=2, padding="same",
                               use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)

    # 8->16
    x = layers.Conv2DTranspose(base_ch*2, kernel_size=4, strides=2, padding="same",
                               use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)

    # 16->32
    x = layers.Conv2DTranspose(base_ch, kernel_size=4, strides=2, padding="same",
                               use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)

    # 32->64
    x = layers.Conv2DTranspose(channels, kernel_size=4, strides=2, padding="same",
                               use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    out = layers.Activation("tanh")(x)
    return keras.Model(z_in, out, name="G_dcgan")

def make_discriminator(img_size=IMG_SIZE, channels=CHANNELS, base_ch=BASE_CH):
    assert img_size == 64, "This template expects IMG_SIZE=64 (change model if needed)."
    x_in = layers.Input(shape=(img_size, img_size, channels))

    x = layers.Conv2D(base_ch, kernel_size=4, strides=2, padding="same",
                      kernel_initializer=WEIGHT_INIT)(x_in)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(base_ch*2, kernel_size=4, strides=2, padding="same",
                      use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(base_ch*4, kernel_size=4, strides=2, padding="same",
                      use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(base_ch*8, kernel_size=4, strides=2, padding="same",
                      use_bias=False, kernel_initializer=WEIGHT_INIT)(x)
    x = layers.BatchNormalization()(x); x = layers.LeakyReLU(0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1, kernel_initializer=WEIGHT_INIT)(x)  # logits
    return keras.Model(x_in, x, name="D_dcgan")

tf.keras.backend.clear_session()
with strategy.scope():
    G = make_generator()
    G_ema = make_generator(); G_ema.set_weights(G.get_weights())
    D = make_discriminator()
    gen_opt  = keras.optimizers.Adam(GEN_LR,  beta_1=BETA_1, beta_2=BETA_2)
    disc_opt = keras.optimizers.Adam(DISC_LR, beta_1=BETA_1, beta_2=BETA_2)
bce_logits = keras.losses.BinaryCrossentropy(from_logits=True)
print(G.summary()); print(D.summary())


In [ ]:

# =====================
# Train Steps (non-saturating) + optional instance noise + EMA
# =====================
LATENTS_FIXED = tf.random.normal([SAMPLES, LATENT_DIM])

def ema_update():
    for we, w in zip(G_ema.weights, G.weights):
        we.assign(0.999 * we + 0.001 * w)

@tf.function(jit_compile=True)
def d_step(real_images, sigma):
    b = tf.shape(real_images)[0]
    z = tf.random.normal([b, LATENT_DIM])
    with tf.GradientTape() as tape:
        fake = G(z, training=True)
        if sigma > 0:
            nr = tf.random.normal(tf.shape(real_images), stddev=sigma, dtype=real_images.dtype)
            nf = tf.random.normal(tf.shape(fake),        stddev=sigma, dtype=fake.dtype)
            real_in = tf.clip_by_value(real_images + nr, -1.0, 1.0)
            fake_in = tf.clip_by_value(fake + nf,        -1.0, 1.0)
        else:
            real_in, fake_in = real_images, fake

        real_logits = D(real_in, training=True)
        fake_logits = D(fake_in, training=True)

        real_t = tf.random.uniform(tf.shape(real_logits), REAL_LABEL_MIN, REAL_LABEL_MAX, dtype=real_logits.dtype)
        fake_t = tf.random.uniform(tf.shape(fake_logits), FAKE_LABEL_MIN,  FAKE_LABEL_MAX,  dtype=fake_logits.dtype)

        d_loss = bce_logits(real_t, real_logits) + bce_logits(fake_t, fake_logits)
    grads = tape.gradient(d_loss, D.trainable_variables)
    disc_opt.apply_gradients(zip(grads, D.trainable_variables))
    return d_loss

@tf.function(jit_compile=True)
def g_step():
    b = BATCH_SIZE
    z = tf.random.normal([b, LATENT_DIM])
    with tf.GradientTape() as tape:
        fake = G(z, training=True)
        fake_logits = D(fake, training=True)
        g_loss = bce_logits(tf.ones_like(fake_logits), fake_logits)
    grads = tape.gradient(g_loss, G.trainable_variables)
    gen_opt.apply_gradients(zip(grads, G.trainable_variables))
    ema_update()
    return g_loss

def generate_preview_samples():
    imgs = G_ema(LATENTS_FIXED, training=False)
    imgs = (imgs + 1.0) * 0.5
    return tf.clip_by_value(imgs, 0.0, 1.0).numpy()


In [ ]:

# =====================
# FID wrappers using EMA generator
# =====================
def sample_fake_uint8_from_gan_ema(n, gen_batch=FID_GEN_BATCH):
    outs, left = [], n
    while left > 0:
        m = min(gen_batch, left)
        z = tf.random.normal([m, LATENT_DIM])
        imgs = G_ema(z, training=False)
        imgs = (imgs + 1.0) * 0.5
        outs.append(tf.cast(tf.clip_by_value(imgs, 0.0, 1.0) * 255.0, tf.uint8))
        left -= m
    return tf.concat(outs, 0)

def compute_fid_fast_gan_ema(ds, num_real=REAL_FID_SAMPLES, num_fake=FAKE_FID_SAMPLES,
                             act_batch=FID_ACT_BATCH, gen_batch=FID_GEN_BATCH):
    global REAL_STATS_CACHE
    if REAL_STATS_CACHE is None:
        x = sample_real_uint8(ds, num_real)
        a = _activations_batched(x, batch_size=act_batch)
        REAL_STATS_CACHE = _mu_sigma(a)
    mu_r, s_r = REAL_STATS_CACHE
    x_fake = sample_fake_uint8_from_gan_ema(num_fake, gen_batch=gen_batch)
    a_fake = _activations_batched(x_fake, batch_size=act_batch)
    mu_f, s_f = _mu_sigma(a_fake)
    return _fid(mu_r, s_r, mu_f, s_f)


In [ ]:

# =====================
# Checkpoints
# =====================
ckpt = tf.train.Checkpoint(G=G, G_ema=G_ema, D=D, gen_opt=gen_opt, disc_opt=disc_opt)
manager = tf.train.CheckpointManager(ckpt, CKPT_DIR, max_to_keep=3)
if manager.latest_checkpoint:
    ckpt.restore(manager.latest_checkpoint)
    print(f"Restored from {manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")


In [ ]:

# =====================
# Training Loop + CSV + grids + FID
# =====================
import pandas as pd

def sigma_for_epoch(ep):
    if EPOCHS <= 1: return float(INSTANCE_NOISE_END)
    t = (ep - 1) / (EPOCHS - 1)
    return float((1.0 - t) * INSTANCE_NOISE_START + t * INSTANCE_NOISE_END)

history = []
print("Starting DCGAN training...")
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    step = 0
    last_g, last_d = None, None
    sigma = sigma_for_epoch(epoch)
    for real_batch in train_dataset:
        d_loss = d_step(real_batch, tf.constant(sigma, dtype=real_batch.dtype))
        g_loss = g_step()
        last_g, last_d = g_loss, d_loss
        step += 1
        if step >= STEPS_PER_EPOCH: break
    secs = time.time() - t0

    # grid
    if (epoch % SAVE_GRID_EVERY) == 0:
        imgs_01 = generate_preview_samples()
        save_and_show_grid(imgs_01, epoch, secs)

    fid_val = None
    if COMPUTE_FID_EVERY and (epoch % COMPUTE_FID_EVERY) == 0:
        try:
            t1 = time.time()
            fid_val = compute_fid_fast_gan_ema(train_dataset)
            print(f"FID @ epoch {epoch}: {fid_val:.2f} (took {time.time()-t1:.1f}s)")
        except Exception as e:
            print(f"FID computation failed: {e}")

    if (epoch % 10) == 0:
        path = manager.save(); print(f"Saved checkpoint at {path}")

    row = {"epoch": epoch,
           "g_loss": float(last_g.numpy()) if last_g is not None else np.nan,
           "d_loss": float(last_d.numpy()) if last_d is not None else np.nan,
           "secs": secs, "fid": fid_val}
    history.append(row)
    try:
        pd.DataFrame(history).to_csv(METRICS_CSV, index=False)
    except Exception as e:
        print(f"(Could not save metrics CSV: {e})")

print("Training complete.")


In [ ]:

# =====================
# Plots: Loss & FID
# =====================
import pandas as pd

if os.path.exists(METRICS_CSV):
    df = pd.read_csv(METRICS_CSV)
    if not df.empty:
        # Loss
        plt.figure()
        plt.plot(df["epoch"], df["g_loss"], label="G loss")
        plt.plot(df["epoch"], df["d_loss"], label="D loss")
        plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("GAN Loss vs Epoch"); plt.legend(); plt.show()

        # FID
        if "fid" in df.columns and df["fid"].notna().any():
            df_fid = df.dropna(subset=["fid"])
            plt.figure()
            plt.plot(df_fid["epoch"], df_fid["fid"])
            plt.xlabel("Epoch"); plt.ylabel("FID"); plt.title("FID vs Epoch"); plt.show()
    else:
        print("Metrics CSV is empty.")
else:
    print("Metrics CSV not found.")


In [ ]:

# =====================
# Make training grid GIF from saved sample grids
# =====================
import os, glob, imageio
from IPython.display import display, Image as IPyImage

annot = sorted(glob.glob(os.path.join(SAMPLES_DIR, "epoch_*_annotated.png")))
raw   = sorted(glob.glob(os.path.join(SAMPLES_DIR, "epoch_*.png")))
files = annot if annot else raw
assert files, f"No sample images found under {SAMPLES_DIR}. Run training first."

GIF_PATH   = os.path.join(OUTPUT_DIR, "training_recap.gif")
FRAME_SKIP = 1
DURATION   = 0.15

with imageio.get_writer(GIF_PATH, mode="I", duration=DURATION) as writer:
    for i, f in enumerate(files):
        if (i % FRAME_SKIP) != 0:
            continue
        writer.append_data(imageio.imread(f))

display(IPyImage(filename=GIF_PATH, width=DISPLAY_WIDTH_PX))
print(f"Saved GIF: {GIF_PATH}  |  frames: {len(files[::FRAME_SKIP])}  |  duration/frame: {DURATION}s")


In [ ]:

# =====================
# FID-vs-Epoch GIF (one frame per epoch; forward-fill missing)
# =====================
import os, shutil, imageio, numpy as np, pandas as pd, matplotlib.pyplot as plt
from IPython.display import Image as IPyImage, display

metrics_csv = METRICS_CSV if 'METRICS_CSV' in globals() else "metrics_history.csv"
out_dir     = OUTPUT_DIR   if 'OUTPUT_DIR'   in globals() else "."
frames_dir  = os.path.join(out_dir, "fid_frames_tmp")
gif_path    = os.path.join(out_dir, "fid_progress.gif")
frame_dur_s = 0.12
keep_frames = False

assert os.path.exists(metrics_csv), f"Metrics CSV not found: {metrics_csv}"
df = pd.read_csv(metrics_csv)
assert "epoch" in df.columns, "CSV missing 'epoch' column"
max_epoch = int(df["epoch"].max())

epochs_full = np.arange(1, max_epoch + 1, dtype=int)
fid_series = df.set_index("epoch")["fid"].reindex(epochs_full).ffill()

valid = fid_series.dropna()
if len(valid) == 0:
    y_min, y_max = 100.0, 400.0
else:
    pad = 0.1 * (valid.max() - valid.min() + 1e-6)
    y_min = float(valid.min() - pad)
    y_max = float(valid.max() + pad)

os.makedirs(frames_dir, exist_ok=True)
for e in epochs_full:
    plt.figure(figsize=(6, 4), dpi=120)
    plt.plot(epochs_full[:e], fid_series.values[:e])
    plt.xlabel("Epoch"); plt.ylabel("FID")
    plt.title(f"FID vs Epoch  ·  Epoch {e}/{max_epoch}")
    plt.ylim(y_min, y_max); plt.xlim(1, max_epoch)
    plt.grid(True, alpha=0.25)
    frame_path = os.path.join(frames_dir, f"fid_{e:04d}.png")
    plt.savefig(frame_path, bbox_inches="tight")
    plt.close()

with imageio.get_writer(gif_path, mode="I", duration=frame_dur_s) as writer:
    for e in epochs_full:
        writer.append_data(imageio.imread(os.path.join(frames_dir, f"fid_{e:04d}.png")))

if not keep_frames:
    shutil.rmtree(frames_dir, ignore_errors=True)

display(IPyImage(filename=gif_path, width=480))
print(f"Saved FID GIF → {gif_path}  |  frames: {max_epoch}  |  duration/frame: {frame_dur_s}s")


In [ ]:

# =====================
# Side-by-side MP4: training grid GIF + FID GIF (Slides-friendly)
# =====================
# If MoviePy isn't installed, uncomment:
# !pip -q install moviepy imageio-ffmpeg

import moviepy.editor as mpy
from moviepy.video.fx.all import loop as mp_loop
from base64 import b64encode
from IPython.display import HTML

gif_left  = os.path.join(OUTPUT_DIR, "training_recap.gif")
gif_right = os.path.join(OUTPUT_DIR, "fid_progress.gif")
assert os.path.exists(gif_left),  f"Missing: {gif_left}"
assert os.path.exists(gif_right), f"Missing: {gif_right}"

clip_l = mpy.VideoFileClip(gif_left)
clip_r = mpy.VideoFileClip(gif_right)

TARGET_H = 360
clip_l = clip_l.resize(height=TARGET_H)
clip_r = clip_r.resize(height=TARGET_H)

dur = max(clip_l.duration, clip_r.duration)
clip_l = mp_loop(clip_l, duration=dur)
clip_r = mp_loop(clip_r, duration=dur)

final = mpy.clips_array([[clip_l, clip_r]])

fps_l = getattr(clip_l, "fps", None) or 12
fps_r = getattr(clip_r, "fps", None) or 12
fps = max(int(fps_l), int(fps_r), 12)

out_mp4 = os.path.join(OUTPUT_DIR, "grid_plus_fid_side_by_side.mp4")
final.write_videofile(out_mp4, codec="libx264", fps=fps, audio=False, preset="medium")

clip_l.close(); clip_r.close(); final.close()

# Inline preview
with open(out_mp4, "rb") as f:
    mp4_bytes = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4_bytes).decode()
display(HTML(f'<video src="{data_url}" width="800" controls loop muted></video>'))
print(f"Saved video → {out_mp4}\nUpload to Google Drive, then Insert → Video in Google Slides.")
